  ## ALMotion - joint control API


### ALMotion.getAngles("name", sensorvalue[bool])

Gets the angles of  the joints
> __Parameters:__	
- __names__ : names the joints, chains, “Body”, “JointActuators”, “Joints” or “Actuators”.
- __useSensors__ : If true, sensor angles will be returned.
    
> __Returns:__	
- __Joint angles__ in __radians__.

### ALMotion.setAngles("name", anlges[float], fractionMaxspeed[float])

Sets the angles of  the joints. This is a non-block call. (절대각도)
> __Parameters:__	
- __names__ : The name or names of joints, chains, “Body”, “JointActuators”, “Joints” or “Actuators”.
- __angles__ : One or more angles in radians
- __fractionMaxspeed__ : The fraction of maximum speed to use
– 


### ALMotion.changeAngles(names, changes, fractionMaxSpeed)

Changes Angles. This is a non-blocking call. (상대각도)
>__Parameters:__	
 -  __names__ : The name or names of joints, chains, “Body”, “JointActuators”, “Joints” or “Actuators”.
 -  __changes__ : One or more changes in radians
 -  __fractionMaxSpeed__ : The fraction of maximum speed to use


### ALMotion.closeHand(handName)

loses the hand, then cuts motor current to conserve energy. This is a blocking call.
>__Parameters:__	
 -  __handName__ : The name of the hand. Could be: “RHand” or “LHand”.

### ALMotion.openHand(handName)

Opens the hand, then cuts motor current to conserve energy. This is a blocking call.
>__Parameters:__	
 -  __handName__ : The name of the hand. Could be: “RHand” or “LHand”.

### forward_kinematics.py

In [ ]:
import math
import numpy as np
import scipy as sp
from scipy import linalg

# (Unit metter)
L1 = 0.14974 # ShoulderOffsetY (Torso to shoulder y)
L2 = 0.015   # ElbowOffsetY (Shoulder to Elbow y)
L3 = 0.1812  # UpperArmLength (Shoulder to Elobw x)
L4 = 0       # Elbow has two joints (2-DOF)
L5 = 0.150   # LowerArmLength (Elbow to Hand)
L6 = 0.0695  # HandOffsetX (Hand to finger)
L7 = 0.0303  # HandOffsetZ (finger to fingertip)

p = np.array([0,0,0,1])
v0 = np.array([[1],[0],[0],[0]])
v1 = np.array([[0],[1],[0],[0]])
v2 = np.array([[0],[0],[1],[0]])


def transX(th, x, y, z): # roll
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[1, 0, 0, x], [0, c, -s, y], [0, s, c, z], [0, 0, 0, 1]])

def transY(th, x, y, z): # pitch 
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, 0, -s, x], [0, 1, 0, y], [s, 0, c, z], [0, 0, 0, 1]])

def transZ(th, x, y, z): # yaw
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, -s, 0, x], [s, c, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])

In [1]:
def calc_fk_and_jacob(angles, jacob=True, right=True):
# angles = "LShoulderPitch", "LShoulderRoll", "LElbowYaw", "LElbowRoll", "LWristYaw"
    
    _L1_ = -L1 if right else L1
    _L2_ = -L2 if right else L2

    T1 = transY(-angles[0], 0, _L1_, 0)          # y(pitch) Axis rotaion "LShoulderPitch"
    T2 = transZ(angles[1], 0, 0, 0)              # "LSholderRoll"
    Td = transY(9.0/180.0*math.pi, L3, _L2_, 0)  # Offset of elbew 9º and sholuder_offset & UpperArm
    T3 = transX(angles[2], 0, 0, 0)              # "LElbowYaw"
    T4 = transZ(angles[3], 0, 0, 0)              # "LElbowRoll"
    T5 = transX(angles[4], L5, 0, 0)             # "LWristYaw" and LowerArmLength (Elbow to Hand)
    T6 = transZ(0, L6, 0, -L7)                   # HandOffsetX (Hand to finger) and HandOffsetZ (finger to fingertip)
    
    T1Abs = T1
    T2Abs = T1Abs.dot(T2)
    TdAbs = T2Abs.dot(Td)
    T3Abs = TdAbs.dot(T3)
    T4Abs = T3Abs.dot(T4)
    T5Abs = T4Abs.dot(T5)
    T6Abs = T5Abs.dot(T6) # End-effector's Transformation 4 by 4 matrix

    pos = T6Abs.dot(p) # x, y, z 의 translation의 값과 1로 -> [Px, Py, Pz, 1]
    ori = T6Abs[0:3,0:3] # 3 x 3 의 rotaion matrix

    if not jacob:
        return pos, ori

    OfstT1 = L1 * T1Abs.dot(v1) # 
    OfstTd = TdAbs.dot(np.array([[L3], [L2], [0], [0]]))
    OfstT5 = L5 * T5Abs.dot(v0)
    OfstT6 = T5Abs.dot(np.array([[L6], [0], [-L7], [0]]))

    vec6 = OfstT6
    vec5 = vec6 + OfstT5
    vec4 = vec5
    vec3 = vec4
    vecd = vec3 + OfstTd
    vec2 = vecd
    vec1 = vec2 + OfstT1
    
    j1 = T1Abs.dot(v1)
    j2 = T2Abs.dot(v2)
    jd = TdAbs.dot(v1)
    j3 = T3Abs.dot(v0)
    j4 = T4Abs.dot(v2)
    j5 = T5Abs.dot(v0)
    
    J1 = cross(j1, vec1)
    J2 = cross(j2, vec2)
    J3 = cross(j3, vec3)
    J4 = cross(j4, vec4)
    J5 = cross(j5, vec5)
    
    Jv = np.c_[J1, J2, J3, J4, J5]
    Jw = np.c_[j1,j2,j3,j4,j5][0:3,:]
    return pos, ori, Jv, Jw


def cross(j, v):
    t0 = j[1][0] * v[2][0] - j[2][0] * v[1][0]
    t1 = j[2][0] * v[0][0] - j[0][0] * v[2][0]
    t2 = j[0][0] * v[1][0] - j[1][0] * v[0][0]
    return np.array([[t0], [t1], [t2]])
        

In [6]:
import math
import numpy as np
import scipy as sp
from scipy import linalg


angles = [0.8, -0.2, 1.57-0.2, 0.9, -1.57]
angs = np.array([a for a in angles])

L1 = 0.14974 # ShoulderOffsetY (Torso to shoulder y)
L2 = 0.015   # ElbowOffsetY (Shoulder to Elbow y)
L3 = 0.1812  # UpperArmLength (Shoulder to Elobw x)
L4 = 0       # Elbow has two joints (2-DOF)
L5 = 0.150   # LowerArmLength (Elbow to Hand)
L6 = 0.0695  # HandOffsetX (Hand to finger)
L7 = 0.0303  # HandOffsetZ (finger to fingertip)

p = np.array([0,0,0,1])
v0 = np.array([[1],[0],[0],[0]])
v1 = np.array([[0],[1],[0],[0]])
v2 = np.array([[0],[0],[1],[0]])

def transX(th, x, y, z): # roll
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[1, 0, 0, x], [0, c, -s, y], [0, s, c, z], [0, 0, 0, 1]])

def transY(th, x, y, z): # pitch 
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, 0, -s, x], [0, 1, 0, y], [s, 0, c, z], [0, 0, 0, 1]])

def transZ(th, x, y, z): # yaw
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, -s, 0, x], [s, c, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])

_L1_ = L1
_L2_ = L2

T1 = transY(-angles[0], 0, _L1_, 0)          # y(pitch) Axis rotaion "LShoulderPitch"
T2 = transZ(angles[1], 0, 0, L0)              # "LSholderRoll"
Td = transY(9.0/180.0*math.pi, L3, _L2_, 0)  # Offset of elbew 9º and sholuder_offset & UpperArm
T3 = transX(angles[2], 0, 0, 0)              # "LElbowYaw"
T4 = transZ(angles[3], 0, 0, 0)              # "LElbowRoll"
T5 = transX(angles[4], L5, 0, 0)             # "LWristYaw" and LowerArmLength (Elbow to Hand)
T6 = transZ(0, L6, 0, -L7)                   # HandOffsetX (Hand to finger) and HandOffsetZ (finger to fingertip)

T1Abs = T1
T2Abs = T1Abs.dot(T2)
TdAbs = T2Abs.dot(Td)
T3Abs = TdAbs.dot(T3)
T4Abs = T3Abs.dot(T4)
T5Abs = T4Abs.dot(T5)
T6Abs = T5Abs.dot(T6) # End-effector's Transformation 4 by 4 matrix

OfstT1 = L1 * T1Abs.dot(v1) # 
OfstTd = TdAbs.dot(np.array([[L3], [L2], [0], [0]]))
OfstT5 = L5 * T5Abs.dot(v0)
OfstT6 = T5Abs.dot(np.array([[L6], [0], [-L7], [0]]))

vec6 = OfstT6
vec5 = vec6 + OfstT5
vec4 = vec5
vec3 = vec4
vecd = vec3 + OfstTd
vec2 = vecd
vec1 = vec2 + OfstT1

j1 = T1Abs.dot(v1)
j2 = T2Abs.dot(v2)
jd = TdAbs.dot(v1)
j3 = T3Abs.dot(v0)
j4 = T4Abs.dot(v2)
j5 = T5Abs.dot(v0)    #회전축의 벡터

def cross(j, v):
    t0 = j[1][0] * v[2][0] - j[2][0] * v[1][0]
    t1 = j[2][0] * v[0][0] - j[0][0] * v[2][0]
    t2 = j[0][0] * v[1][0] - j[1][0] * v[0][0]
    return np.array([[t0], [t1], [t2]])

J1 = cross(j1, vec1)
J2 = cross(j2, vec2)
J3 = cross(j3, vec3)
J4 = cross(j4, vec4)
J5 = cross(j5, vec5)

Jv = np.c_[J1, J2, J3, J4, J5]
Jw = np.c_[j1,j2,j3,j4,j5][0:3,:]
print(Jw)


epsilon = 0.0001
u, sigma, v = np.linalg.svd(J, full_matrices=True)
sigma_ = [1/s if s > epsilon else 0 for s in sigma]
rank_v = np.shape(J)[0]
rank_h = np.shape(J)[1]


J_i = np.matrix(v.transpose()) * np.matrix(linalg.diagsvd(sigma_, rank_h, rank_v)) * np.matrix(u.transpose())


[[ 0.          0.71735609  0.78663154 -0.01562279  0.97246699]
 [ 1.          0.         -0.19622338 -0.9541765   0.05500131]
 [ 0.          0.69670671 -0.58541199  0.29883631  0.22645709]]
(6, 5)
[[ 0.66327653  0.          0.          0.          0.        ]
 [ 0.          0.70279275  0.          0.          0.        ]
 [ 0.          0.          1.00387463  0.          0.        ]
 [ 0.          0.          0.          8.06910217  0.        ]
 [ 0.          0.          0.          0.         10.30724366]
 [ 0.          0.          0.          0.          0.        ]]
